<a href="https://colab.research.google.com/github/YoungMin-Jang/store/blob/main/%EC%A4%91%EA%B3%A0%EA%B1%B0%EB%9E%98_%EC%99%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 설치


In [1]:
!pip install Flask Flask-Bcrypt Flask-SQLAlchemy Flask-SocketIO pyngrok nest_asyncio google-cloud-storage flask_sqlalchemy flask_login


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install apscheduler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 2.8 MB/s eta 0:00:00


In [6]:
!ngrok config add-authtoken 2q3VcyYC1gvXoksRxJ7tLQUz0aN_2pjntwtm8DUbwvggvGnGK


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## 메인 코드

In [7]:
import shutil
from datetime import datetime
from apscheduler.schedulers.background import BackgroundScheduler

# 데이터베이스 백업 함수
def backup_database():
    source_db_path = '/content/drive/MyDrive/marketplace.db'
    backup_folder = '/content/drive/MyDrive/marketplace_backups'

    # 백업 폴더가 없다면 생성
    if not os.path.exists(backup_folder):
        os.makedirs(backup_folder)

    # 백업 파일 이름에 현재 날짜와 시간을 포함
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_db_path = os.path.join(backup_folder, f'marketplace_backup_{timestamp}.db')

    # 데이터베이스 파일 복사
    shutil.copy2(source_db_path, backup_db_path)
    print(f"Database backup created at: {backup_db_path}")

# 백그라운드 스케줄러 설정
scheduler = BackgroundScheduler()
scheduler.add_job(backup_database, 'interval', minutes=10)  # 10분마다 백업 실행
scheduler.start()


In [ ]:
from flask import Flask, render_template_string, request, redirect, url_for, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_socketio import SocketIO
from pyngrok import ngrok
import nest_asyncio
import os
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive', force_remount=True)

# Colab에서 Flask 실행을 위한 설정
nest_asyncio.apply()

# Flask 애플리케이션 설정
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////content/drive/MyDrive/marketplace.db'
app.config['SECRET_KEY'] = 'your_secret_key'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
socketio = SocketIO(app)

# Flask-Login 설정
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

# 고정 카테고리 및 상태 목록
CATEGORIES = ['전자기기', '도서', '의류', '생활물품', '장난감']
CONDITIONS = ['새상품', '중고']

# 사용자 모델
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password_hash = db.Column(db.String(128), nullable=False)

# 물품 모델
class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    category = db.Column(db.String(80), nullable=False)
    condition = db.Column(db.String(20), nullable=False)  # 새 상품 또는 중고 상품
    price = db.Column(db.Float, nullable=False)
    description = db.Column(db.String(200), nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

# 데이터베이스 생성
with app.app_context():
    db.create_all()


# 사용자 로딩 함수
@login_manager.user_loader
def load_user(user_id):
    return db.session.get(User, int(user_id))

# 회원가입 라우트
@app.route('/register', methods=['POST'])
def register():
    email = request.form['email']
    password = request.form['password']
    existing_user = User.query.filter_by(email=email).first()
    if existing_user:
        return "Email already registered. Please use a different email."

    hashed_password = bcrypt.generate_password_hash(password).decode('utf-8')
    new_user = User(email=email, password_hash=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return redirect(url_for('index'))

# 로그인 라우트
@app.route('/login', methods=['POST'])
def login():
    email = request.form['email']
    password = request.form['password']
    user = User.query.filter_by(email=email).first()
    if user and bcrypt.check_password_hash(user.password_hash, password):
        login_user(user)
        return redirect(url_for('index'))
    return "Invalid credentials. Please try again."

# 로그아웃 라우트
@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('index'))

# 홈 페이지 라우트
@app.route('/')
def index():
    items = Item.query.all()
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>중고마켓</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
        <style>
            body {
                background-color: #f8f9fa;
            }
            .container {
                margin-top: 50px;
            }
            .card {
                margin-bottom: 20px;
            }
            .btn-primary, .btn-success {
                margin-right: 10px;
            }
            .form-select, .form-control {
                margin-bottom: 15px;
            }
            h1, h2 {
                margin-top: 30px;
                margin-bottom: 20px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1 class="text-center">중고마켓에 오신 것을 환영합니다!</h1>

            {% if current_user.is_authenticated %}
                <div class="alert alert-info text-center">
                    <p>환영합니다, <strong>{{ current_user.email }}</strong>님!</p>
                    <a href="/logout" class="btn btn-danger">로그아웃</a>
                </div>

                <h2>물품 추가</h2>
                <form action="/add_item" method="post" class="card p-4 shadow-sm">
                    <input type="text" name="name" class="form-control" placeholder="물품 이름" required>
                    <select name="category" class="form-select" required>
                        <option value="" disabled selected>카테고리 선택</option>
                        {% for cat in categories %}
                            <option value="{{ cat }}">{{ cat }}</option>
                        {% endfor %}
                    </select>
                    <select name="condition" class="form-select" required>
                        <option value="" disabled selected>상태 선택</option>
                        {% for cond in conditions %}
                            <option value="{{ cond }}">{{ cond }}</option>
                        {% endfor %}
                    </select>
                    <input type="number" name="price" class="form-control" placeholder="가격" step="0.01" required>
                    <input type="text" name="description" class="form-control" placeholder="설명" required>
                    <button type="submit" class="btn btn-success">물품 추가</button>
                </form>

                <h2>물품 검색</h2>
                <form action="/search" method="get" class="card p-4 shadow-sm">
                    <input type="text" name="query" class="form-control" placeholder="물품 이름으로 검색" required>
                    <button type="submit" class="btn btn-primary">이름으로 검색</button>
                </form>

                <h2>카테고리로 검색</h2>
                <form action="/search_category" method="get" class="card p-4 shadow-sm">
                    <select name="category" class="form-select" required>
                        <option value="" disabled selected>카테고리 선택</option>
                        {% for cat in categories %}
                            <option value="{{ cat }}">{{ cat }}</option>
                        {% endfor %}
                    </select>
                    <button type="submit" class="btn btn-primary">카테고리로 검색</button>
                </form>

                <h2>상태로 검색</h2>
                <form action="/search_condition" method="get" class="card p-4 shadow-sm">
                    <select name="condition" class="form-select" required>
                        <option value="" disabled selected>상태 선택</option>
                        {% for cond in conditions %}
                            <option value="{{ cond }}">{{ cond }}</option>
                        {% endfor %}
                    </select>
                    <button type="submit" class="btn btn-primary">상태로 검색</button>
                </form>

                <h2>등록된 물품 목록</h2>
                <div class="row">
                    {% for item in items %}
                        <div class="col-md-4">
                            <div class="card shadow-sm">
                                <div class="card-body">
                                    <h5 class="card-title">{{ item.name }}</h5>
                                    <p class="card-text">
                                        <strong>카테고리:</strong> {{ item.category }}<br>
                                        <strong>상태:</strong> {{ item.condition }}<br>
                                        <strong>가격:</strong> ₩{{ "{:,.2f}".format(item.price) }}
                                    </p>
                                    <a href="/recommend/{{ item.id }}" class="btn btn-outline-info">비슷한 물품 추천</a>
                                </div>
                            </div>
                        </div>
                    {% endfor %}
                </div>

            {% else %}
                <h2>회원가입</h2>
                <form action="/register" method="post" class="card p-4 shadow-sm">
                    <input type="email" name="email" class="form-control" placeholder="이메일" required>
                    <input type="password" name="password" class="form-control" placeholder="비밀번호" required>
                    <button type="submit" class="btn btn-success">가입하기</button>
                </form>

                <h2>로그인</h2>
                <form action="/login" method="post" class="card p-4 shadow-sm">
                    <input type="email" name="email" class="form-control" placeholder="이메일" required>
                    <input type="password" name="password" class="form-control" placeholder="비밀번호" required>
                    <button type="submit" class="btn btn-primary">로그인</button>
                </form>
            {% endif %}
        </div>

        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
    </body>
    </html>
    """, categories=CATEGORIES, conditions=CONDITIONS, items=items)


# 물품 등록 라우트 (로그인 필요)
@app.route('/add_item', methods=['POST'])
@login_required
def add_item():
    name = request.form['name']
    category = request.form['category']
    condition = request.form['condition']
    price = float(request.form['price'])
    description = request.form['description']
    new_item = Item(name=name, category=category, condition=condition, price=price, description=description, user_id=current_user.id)
    db.session.add(new_item)
    db.session.commit()
    return redirect(url_for('index'))

# 물품 목록 라우트
@app.route('/items')
def get_items():
    items = Item.query.all()
    return jsonify([
        {
            "name": item.name,
            "category": item.category,
            "condition": item.condition,
            "price": item.price,
            "description": item.description
        }
        for item in items
    ])

# 이름으로 물품 검색 라우트
@app.route('/search')
def search():
    query = request.args.get('query')
    results = Item.query.filter(Item.name.contains(query)).all()
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>검색 결과</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
        <style>
            body {
                background-color: #f8f9fa;
            }
            .container {
                margin-top: 50px;
            }
            .card {
                margin-bottom: 20px;
            }
            h1 {
                margin-bottom: 30px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1 class="text-center">검색 결과</h1>

            {% if results %}
                <div class="row">
                    {% for item in results %}
                        <div class="col-md-4">
                            <div class="card shadow-sm">
                                <div class="card-body">
                                    <h5 class="card-title">{{ item.name }}</h5>
                                    <p class="card-text">
                                        <strong>카테고리:</strong> {{ item.category }}<br>
                                        <strong>상태:</strong> {{ item.condition }}<br>
                                        <strong>가격:</strong> ₩{{ "{:,.2f}".format(item.price) }}<br>
                                        <strong>설명:</strong> {{ item.description }}
                                    </p>
                                </div>
                            </div>
                        </div>
                    {% endfor %}
                </div>
            {% else %}
                <p class="text-center">검색 결과가 없습니다.</p>
            {% endif %}

            <div class="text-center mt-4">
                <a href="{{ url_for('index') }}" class="btn btn-primary">홈으로 돌아가기</a>
            </div>
        </div>

        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
    </body>
    </html>
    """, results=results)


# 카테고리로 물품 검색 라우트
@app.route('/search_category')
def search_category():
    category = request.args.get('category')
    results = Item.query.filter_by(category=category).all()
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>카테고리별 검색 결과</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    </head>
    <body>
        <div class="container mt-5">
            <h1 class="text-center">카테고리: {{ category }}</h1>

            {% if results %}
                <div class="row">
                    {% for item in results %}
                        <div class="col-md-4">
                            <div class="card shadow-sm mb-4">
                                <div class="card-body">
                                    <h5 class="card-title">{{ item.name }}</h5>
                                    <p class="card-text">
                                        <strong>상태:</strong> {{ item.condition }}<br>
                                        <strong>가격:</strong> ₩{{ "{:,.2f}".format(item.price) }}<br>
                                        <strong>설명:</strong> {{ item.description }}
                                    </p>
                                </div>
                            </div>
                        </div>
                    {% endfor %}
                </div>
            {% else %}
                <p class="text-center">이 카테고리에는 물품이 없습니다.</p>
            {% endif %}
        </div>
    </body>
    </html>
    """, results=results, category=category)


# 상태로 물품 검색 라우트
@app.route('/search_condition')
def search_condition():
    condition = request.args.get('condition')
    results = Item.query.filter_by(condition=condition).all()
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>상태별 검색 결과</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    </head>
    <body>
        <div class="container mt-5">
            <h1 class="text-center">상태: {{ condition }}</h1>

            {% if results %}
                <div class="row">
                    {% for item in results %}
                        <div class="col-md-4">
                            <div class="card shadow-sm mb-4">
                                <div class="card-body">
                                    <h5 class="card-title">{{ item.name }}</h5>
                                    <p class="card-text">
                                        <strong>카테고리:</strong> {{ item.category }}<br>
                                        <strong>가격:</strong> ₩{{ "{:,.2f}".format(item.price) }}<br>
                                        <strong>설명:</strong> {{ item.description }}
                                    </p>
                                </div>
                            </div>
                        </div>
                    {% endfor %}
                </div>
            {% else %}
                <p class="text-center">이 상태의 물품이 없습니다.</p>
            {% endif %}
        </div>
    </body>
    </html>
    """, results=results, condition=condition)

# 추천 시스템 라우트 (로그인 필요)
@app.route('/recommend/<int:item_id>')
@login_required
def recommend(item_id):
    # 기준 물품 가져오기
    base_item = Item.query.get_or_404(item_id)

    # 추천 기준: 같은 카테고리, 비슷한 가격(±20%), 같은 상태
    min_price = base_item.price * 0.8
    max_price = base_item.price * 1.2

    recommendations = Item.query.filter(
        Item.id != base_item.id,
        Item.category == base_item.category,
        Item.condition == base_item.condition,
        Item.price.between(min_price, max_price)
    ).all()

    # 추천 결과를 HTML로 렌더링
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>추천 물품</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
        <style>
            body {
                background-color: #f8f9fa;
            }
            .container {
                margin-top: 50px;
            }
            .card {
                margin-bottom: 20px;
            }
            h1 {
                margin-bottom: 30px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1 class="text-center">추천 물품</h1>
            <h3 class="text-center">기준 물품: <strong>{{ base_item.name }}</strong></h3>

            {% if recommendations %}
                <div class="row">
                    {% for item in recommendations %}
                        <div class="col-md-4">
                            <div class="card shadow-sm">
                                <div class="card-body">
                                    <h5 class="card-title">{{ item.name }}</h5>
                                    <p class="card-text">
                                        <strong>카테고리:</strong> {{ item.category }}<br>
                                        <strong>상태:</strong> {{ item.condition }}<br>
                                        <strong>가격:</strong> ₩{{ "{:,.2f}".format(item.price) }}<br>
                                        <strong>설명:</strong> {{ item.description }}
                                    </p>
                                </div>
                            </div>
                        </div>
                    {% endfor %}
                </div>
            {% else %}
                <p class="text-center">추천할 물품이 없습니다.</p>
            {% endif %}

            <div class="text-center mt-4">
                <a href="{{ url_for('index') }}" class="btn btn-primary">홈으로 돌아가기</a>
            </div>
        </div>

        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
    </body>
    </html>
    """, base_item=base_item, recommendations=recommendations)

# Flask 서버 실행
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel: {public_url}")

socketio.run(app, port=5000, allow_unsafe_werkzeug=True)


Mounted at /content/drive


 * ngrok tunnel: https://c7a0-34-139-103-76.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:33:33] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:33:34] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:33:40] "POST /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:33:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:33:50] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:33:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:33:58] "GET /search_category?category=전자기기 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:34:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:34:24] "POST /add_item HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 04:34:25] "GET / HTTP/1.1" 2